In [1]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-gdgaj6pn
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-gdgaj6pn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for flair: filename=flair-0.4.5-cp36-none-any.whl size=148505 sha256=de8f8acddd6a187c403f45e87a8ae50aa436c4008131c08305cb3ba0c1772eb0
  Stored in directory: /tmp/pip-ephem-wheel-cache-dl7yk0qi/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Found existing installation: flair 0.4.5
    Uninstalling flair-0.4.5:
      Successfully uninstalled flair-0.4.5


In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_folder = "./drive/My Drive/capstone/data/"

## Train on benchmark dataset

In [0]:
benchmark = pd.read_csv(data_folder + "all-data.csv",encoding = "ISO-8859-1", header=None)

In [6]:
benchmark.head()

,0,1
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [0]:
benchmark = benchmark[[0, 1]].rename(columns={0:"label", 1:"text"})

In [0]:
def convert_label(label):
  'convert label from text to -1, 0, and 1.'
    
  if label == 'neutral':
      return 0
  elif label == 'negative':
      return -1
  else:
      return 1

benchmark['label'] = benchmark['label'].apply(convert_label)

In [9]:
benchmark.head()

,label,text
0,0,"According to Gran , the company has no plans t..."
1,0,Technopolis plans to develop in stages an area...
2,-1,The international electronic industry company ...
3,1,With the new production plant the company woul...
4,1,According to the company 's updated strategy f...


In [10]:
#benchmark['label'] = '__label__' + benchmark['label'].astype(str)
#benchmark.head()

,label,text
0,__label__0,"According to Gran , the company has no plans t..."
1,__label__0,Technopolis plans to develop in stages an area...
2,__label__-1,The international electronic industry company ...
3,__label__1,With the new production plant the company woul...
4,__label__1,According to the company 's updated strategy f...


#### Create train, dev and test set

In [0]:
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

#### Build corpus

In [15]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
)

2020-05-21 16:26:20,848 Reading data from drive/My Drive/capstone/data
2020-05-21 16:26:20,850 Train: drive/My Drive/capstone/data/train.csv
2020-05-21 16:26:20,851 Dev: drive/My Drive/capstone/data/dev.csv
2020-05-21 16:26:20,853 Test: drive/My Drive/capstone/data/test.csv


#### Create word embeddings

In [17]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.


In [18]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.


2020-05-21 16:29:35,472 Computing label dictionary. Progress:


100%|██████████| 4359/4359 [00:03<00:00, 1112.83it/s]

2020-05-21 16:29:39,766 [b'0', b'-1', b'1']
2020-05-21 16:29:39,788 ----------------------------------------------------------------------------------------------------
2020-05-21 16:29:39,794 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-05-21 16:29:50,593 epoch 1 - iter 12/122 - loss 1.09119564 - samples/sec: 38.15
2020-05-21 16:30:14,205 epoch 1 - iter 24/122 - loss 0.97279288 - samples/sec: 38.31
2020-05-21 16:30:38,825 epoch 1 - iter 36/122 - loss 0.91534476 - samples/sec: 35.48
2020-05-21 16:31:03,061 epoch 1 - iter 48/122 - loss 0.89804776 - samples/sec: 36.38
2020-05-21 16:31:26,959 epoch 1 - iter 60/122 - loss 0.87090534 - samples/sec: 37.57
2020-05-21 16:31:51,281 epoch 1 - iter 72/122 - loss 0.83535755 - samples/sec: 36.86
2020-05-21 16:32:15,061 epoch 1 - iter 84/122 - loss 0.81772747 - samples/sec: 38.12
2020-05-21 16:32:38,666 epoch 1 - iter 96/122 - loss 0.80579187 - samples/sec: 38.81
2020-05-21 16:33:02,459 epoch 1 - iter 108/122 - loss 0.78620375 - samples/sec: 37.99
2020-05-21 16:33:26,034 epoch 1 - iter 120/122 - loss 0.77099588 - samples/sec: 38.60
2020-05-21 16:33:40,910 ----------------------------------------------------------------------------------------------------
2020-05-21 16:33:40,912

{'dev_loss_history': [2.529153823852539,
  3.1829891204833984,
  3.4412078857421875,
  2.397822618484497,
  4.3872294425964355,
  0.838103175163269,
  2.6835856437683105,
  2.2200350761413574,
  3.7298941612243652,
  4.844223499298096],
 'dev_score_history': [0.4573002754820937,
  0.46143250688705234,
  0.46005509641873277,
  0.4931129476584022,
  0.465564738292011,
  0.8278236914600551,
  0.49586776859504134,
  0.5316804407713499,
  0.48484848484848486,
  0.43526170798898073],
 'test_score': 0.6790633608815427,
 'train_loss_history': [0.7646573711125577,
  0.620830442328922,
  0.5782061121502861,
  0.5436141591091626,
  0.5132139423770494,
  0.5029887211371641,
  0.48485090681275383,
  0.44522049644442857,
  0.4305564264049295,
  0.4298317696227402]}